## Exploratory Research in Neural Networks and SVMs

In [ ]:
import numpy as np
import mnist_reader
import sklearn as sk
import matplotlib.pyplot as plt

## Data Preparation:

In [ ]:
X_train, y_train = mnist_reader.load_mnist('fashion', kind='train')
X_test, y_test = mnist_reader.load_mnist('fashion', kind='t10k')

print(X_train.shape)
print(X_test.shape)

In [ ]:
# Filtering the samples but the ones with labels 5 and 7

mask_train = (y_train == 5) | (y_train ==7)
X_train = X_train[mask_train]
y_train = y_train[mask_train]

mask_test = (y_test == 5) | (y_test ==7)
X_test = X_test[mask_test]
y_test = y_test[mask_test]

In [ ]:
# Changing Label 5 and 7 to 0 and 1 respectively

y_train = np.where(y_train == 5, 0, 1)
y_test = np.where(y_test == 5, 0, 1)

print(X_train.shape)
print(X_test.shape)

In [ ]:
# Normalizing each feature vector to its unit form

epsilon = 1e-10

norm_train = np.linalg.norm(X_train, axis=1, keepdims=True)
norm_train = np.where(norm_train == 0, epsilon, norm_train)
X_train = X_train/norm_train

norm_test = np.linalg.norm(X_test, axis=1, keepdims=True)
norm_test = np.where(norm_test == 0, epsilon, norm_test)
X_test = X_test/norm_test


In [ ]:
# Splitting the current data set to get a validation set

X_train, X_validation, y_train, y_validation = sk.model_selection.train_test_split(X_train, y_train, train_size = 0.5, test_size=0.2, random_state = 1)

print(X_train.shape)
print(X_validation.shape)

In [ ]:
# Adding noise to the data set

noise_mask_train = np.random.rand(len(y_train)) < 0.2
y_train_noisy = np.copy(y_train)
y_train_noisy[noise_mask_train] = np.where(y_train_noisy[noise_mask_train] == 0, 1, 0)

## Methods

In [ ]:
# k-fold cross-validation training function

def k_fold_training(x, y, model, k):

    fold_size = len(x)//k

    indices = np.arange(len(x))

    np.random.shuffle(indices)

    scores = []

    for i in range(k):
        testing_indices = indices[i*fold_size : (i+1)*fold_size]
        training_indices = np.concatenate((indices[:i*fold_size], indices[(i+1)*fold_size:]))

        x_training, x_testing = x[training_indices], x[testing_indices]
        y_training, y_testing = y[training_indices], y[testing_indices]

        model.fit(x_training, y_training)

        accuracy = model.score(x_training, y_training)
        scores.append(accuracy)
        
    return np.mean(scores)

In [ ]:
# Function to train svm from a list of C or Gamma values

def train_svm(c_values = None, gamma_values = None, kernel:str = 'linear'):

    training_scores = []
    validation_scores = []

    if kernel == 'linear':
        for c in c_values:
            linear_svm = sk.svm.SVC(kernel = kernel, C = c)
            linear_svm.fit(X_train, y_train_noisy)
            
            validation_scores.append(linear_svm.score(X_validation, y_validation))
            training_scores.append(linear_svm.score(X_train, y_train))

            print(f'Scores for C={c} already processed')

    elif (kernel == 'gaussian') or (kernel == 'rbf'):
        for gamma in gamma_values:
            gaussian_svm = sk.svm.SVC(kernel = kernel, gamma = gamma)
            gaussian_svm.fit(X_train, y_train_noisy)
            
            validation_scores.append(gaussian_svm.score(X_validation, y_validation))
            training_scores.append(gaussian_svm.score(X_train, y_train))

            print(f'Scores for Gamma={gamma} already processed')

    else:
        raise ValueError("Unsupported kernel. Please use 'linear' or 'gaussian'.")

    return training_scores, validation_scores

In [ ]:
# Fuction that uses "k_fold_training" to compare different values of C or Gamma

def cross_validation(models_list):
    models_scores = []
    count = 0
    for model in models_list:
        models_scores.append(k_fold_training(X_train, y_train_noisy, model, 5))
        count += 1
        print(f'Model number {count} processed')
    return models_scores

In [ ]:
# Testing C or Gamma values

def test_svm(c_values = None, gamma_values = None, kernel:str = 'linear'):

    X_combined = np.vstack((X_train, X_validation))
    y_combined = np.concatenate((y_train, y_validation), axis=0)

    
    training_scores = []
    test_scores = []

    if kernel == 'linear':
        for c in c_values:
            linear_svm = sk.svm.SVC(kernel = kernel, C = c)
            linear_svm.fit(X_combined, y_combined)

            test_scores.append(linear_svm.score(X_test, y_test))
            training_scores.append(linear_svm.score(X_train, y_train))
            
            

            print(f'Scores for C={c} were processed')

    elif (kernel == 'gaussian') or (kernel == 'rbf'):
        for gamma in gamma_values:
            gaussian_svm = sk.svm.SVC(kernel = kernel, gamma = gamma)
            gaussian_svm.fit(X_combined, y_combined)
            
            test_scores.append(gaussian_svm.score(X_test, y_test))
            training_scores.append(gaussian_svm.score(X_train, y_train))

            print(f'Scores for Gamma={gamma} were processed')

    else:
        raise ValueError("Unsupported kernel. Please use 'linear' or 'gaussian'.")

    return training_scores, test_scores

# Experiments and Analysis
**SVM with linear kernel**

In [ ]:
# Preparing the list of C values to be tested

C_values = [0.001]
for val in range(1,10):
    C_values.append(C_values[0]*(4**val))
    
print(C_values)

In [ ]:
# Training SVMs with the list of Cs

training_scores, validation_scores = train_svm(C_values, kernel = 'linear')
print (training_scores)
print (validation_scores)

In [ ]:
# Plotting the error in training and validation sets 

plt.figure(figsize=(10, 6))

plt.plot(C_values, training_scores, label='Training Score', marker='o', linestyle='-', color='blue')
    
plt.plot(C_values, validation_scores, label='Validation Score', marker='o', linestyle='--', color='red')
    

plt.xlabel('C Values')
plt.ylabel('Score')
plt.title('SVM Scores vs C Values')
plt.legend()
plt.grid(True)
plt.xscale('log')
plt.show()

In [ ]:
# Using K-fold training for the best 5 C values

best_Cs = [0.512, 1.024, 2.048, 4.096, 8.192]
models = []
for c in best_Cs:
    linear_svm = sk.svm.SVC(kernel = 'linear', C = c)
    models.append(linear_svm)
    
models_scores = cross_validation(models)

print(models_scores)

In [ ]:
# Plotting the error in training and test sets

plt.figure(figsize=(10, 6))

plt.plot(best_Cs, models_scores, label='Training Score', marker='o', linestyle='-', color='blue')    

plt.xlabel('C Values')
plt.ylabel('Score')
plt.title('SVM Scores vs C Values Using K-fold Cross-Validation')
plt.legend()
plt.grid(True)
plt.xscale('log')
plt.show()

In [ ]:
# Testing Values of C

best_Cs_test = [0.512, 1.024, 2.048, 4.096, 8.192, 16.384]

training_scores, test_scores = test_svm(c_values = best_Cs_test, kernel = 'linear')

print (training_scores, test_scores)

In [ ]:
plt.figure(figsize=(10, 6))

plt.plot(best_Cs_test, training_scores, label='Training Score', marker='o', linestyle='-', color='blue')
plt.plot(best_Cs_test, test_scores, label='Testing Score', marker='o', linestyle='--', color='red')    


plt.xlabel('C Values')
plt.ylabel('Score')
plt.title('SVM Scores vs C Values in the Full Training and Test Sets')
plt.legend()
plt.grid(True)
plt.xscale('log')
plt.show()

**SVM with gaussian kernels**